# **Image Generator를 이용한 자료 증대**
> 개와 고양이 분류하기 (with Generator, 이전학습, Fine tuning)

In [1]:
# 관련 이미지가 들어있는 디렉토리 지정
train_dir = '/content/drive/MyDrive/Colab Notebooks/DL/[STAT433] 딥러닝을 위한 통계적모델링/data/dogs and cats_small/train'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/DL/[STAT433] 딥러닝을 위한 통계적모델링/data/dogs and cats_small/validation'

## ImageDataGenerator 
- RGB, 정규화 등 사전정리가능
- 배치 단위의 이미지 자료 생성기
- 이미지 자료가 디렉토리에 있으면 flow_from_directory, 데이터프레임이면 flow_from_dataframe 함수 이용

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
train_datagen=ImageDataGenerator(rescale=1./255)   # standardization (rescaling)
test_datagen=ImageDataGenerator(rescale=1./255)   ## 뒤에서는 이 부분에 augmentation을 한다!

# .flow_from_directory를 이용 
train_generator=train_datagen.flow_from_directory(directory=train_dir,target_size=(150,150),
                                                  batch_size=20,class_mode='binary')
validation_generator=test_datagen.flow_from_directory(directory=validation_dir,target_size=(150,150),
                                                      batch_size=20,class_mode='binary')
# target_size로 (높이, 너비) 지정 = 픽셀의 크기
# batch_size는 출력할 배치의 '크기'를 지정 (개수가 아님)
# class_mode는 binary data로 지정

Found 2000 images belonging to 2 classes.
Found 1010 images belonging to 2 classes.


각각 train은 2000개, valid는 1000개 정도 되는 작은 데이터이다

In [3]:
# 제너레이터는 튜플을 출력함
for x,y in train_generator:
    print(x.shape)  # (배치, 높이, 너비, 채널)
    print(y.shape)
    break      # 반드시 break를 해야함(안하면 계속 반복됨)
    
#########이 부분은 학습에 필요 없는 부분임. 어차피 트테로 들어가는건 train_generator 객체임 ############

(20, 150, 150, 3)
(20,)


In [10]:
print(x[0].shape, y.shape)

(150, 150, 3) (20,)


In [11]:
y

array([0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
       1., 0., 0.], dtype=float32)

## 1) Simple CNN

In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import models
cat_dog_model=models.Sequential()
cat_dog_model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))  # training_generator에서 지정한 target_size
cat_dog_model.add(MaxPooling2D(2,2))
cat_dog_model.add(Conv2D(64,(3,3),activation='relu'))
cat_dog_model.add(MaxPooling2D(2,2))
cat_dog_model.add(Conv2D(128,(3,3),activation='relu'))
cat_dog_model.add(MaxPooling2D(2,2))
cat_dog_model.add(Flatten())
cat_dog_model.add(Dense(512,activation='relu'))
cat_dog_model.add(Dense(1,activation='sigmoid'))
cat_dog_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 36992)             0

모수가 지금 매우매우 많은 상황이다!

In [ ]:
from tensorflow.keras import optimizers
opt=optimizers.RMSprop(lr=1e-4)
cat_dog_model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['acc'])

## fit_generator method 사용
cat_dog_result=cat_dog_model.fit_generator(train_generator,  
                                           steps_per_epoch=100,  # 1에폭당 몇개의 배치를 쓸 것인지?
                                           # 앞서 지정한 배치의 크기는 20개, 데이터는 2000개므로 200개의 배치
                                           epochs=10,
                                           validation_data=validation_generator,  # validation_generator로 지정
                                           validation_steps=50) 
                                           # 앞서 지정한 배치의 크기 20개, 데이터는 1000개이므로 50개의 배치 

In [ ]:
import matplotlib.pyplot as plt
acc=cat_dog_result.history['acc']
val_acc=cat_dog_result.history['val_acc']
loss=cat_dog_result.history['loss']
val_loss=cat_dog_result.history['val_loss']
epochs=range(1,len(acc)+1)
plt.plot(epochs,acc,'b',label='training acc')
plt.plot(epochs,val_acc,'bo',label='validation acc')
plt.title('Simple CNN training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs,loss,'b',label='training loss')
plt.plot(epochs,val_loss,'bo',label='validation loss')
plt.title('Simple CNN training and validation loss')  # 많이 흔들리는걸 보면 별로 많이 좋지 않음(배치에 의존하고있다는 소리임)
plt.legend()
plt.show()

오버피팅이 극심하게 발생함. 그 이유는 2000개의 자료에서 320만개의 모수를 추론하려고 했기 때문임
> 1) 마지막 dense 층의 모수를 임의로 줄이거나 **regularization(L1)**을 출력층 전에 추가하여 해결 가능      


> 2) 자료를 증대하여 해결 가능

## 2) Data Augmentation
오버피팅 해결을 위한 데이터 증대

In [4]:
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=40, width_shift_range=0.2,  
### 40으로 설정해서 independence를 보장하는 데이터를 만든다. random이 굉장히 중요한 개념이 된다
                               height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                horizontal_flip=True, fill_mode='nearest')  # 빈 곳을 가장 가까운 데이터로 채운다 > 완전히 다른 이미지를 만듦

# valid는 그대로 적용을 해야하니까 generate를 하지 않는다
validation_datagen=ImageDataGenerator(rescale=1./255)  

## 다시 flow_from_directory 이용
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(150,150),   
                                              batch_size=32, class_mode='binary')
validation_generator=train_datagen.flow_from_directory(validation_dir,target_size=(150,150),
                                              batch_size=32, class_mode='binary')  
# 배치사이즈에 의존하기 때문에 train, val의 배치사이즈는 같게하자!
# 앞보다 배치사이즈를 32개 증가시켰다. 

Found 2000 images belonging to 2 classes.
Found 1010 images belonging to 2 classes.


In [ ]:
# 같은 오버피팅 유발 모델 똑같이 적합
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense, Dropout
from tensorflow.keras import models
aug_model=models.Sequential()
aug_model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
aug_model.add(MaxPooling2D(2,2))
aug_model.add(Conv2D(64,(3,3),activation='relu'))
aug_model.add(MaxPooling2D(2,2))
aug_model.add(Conv2D(128,(3,3),activation='relu'))
aug_model.add(MaxPooling2D(2,2))
aug_model.add(Flatten())
aug_model.add(Dropout(0.5))
aug_model.add(Dense(512,activation='relu'))
aug_model.add(Dense(1,activation='sigmoid'))
aug_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 36992)            

In [ ]:
from tensorflow.keras import optimizers
aug_model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])

aug_result=cat_dog_model.fit_generator(train_generator,
                                       steps_per_epoch=100,
                                       epochs=10,
                                       validation_data=validation_generator,
                                       validation_steps=31)

In [ ]:
import matplotlib.pyplot as plt
acc=aug_result.history['acc']
val_acc=aug_result.history['val_acc']
loss=aug_result.history['loss']
val_loss=aug_result.history['val_loss']
epochs=range(1,len(acc)+1)
plt.plot(epochs,acc,'b',label='training acc')
plt.plot(epochs,val_acc,'bo',label='validation acc')
plt.title('training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs,loss,'b',label='training loss')
plt.plot(epochs,val_loss,'bo',label='validation loss')
plt.title('training and validation loss')
plt.legend()
plt.show()   # 아직 많이 흔들리는걸로 보아 문제가 많이 있다

자료증대를 통해 independent한 데이터를 늘림으로써 error를 추가해준 것임.      

이는 uncertainty를 늘려 모형을 general 하게 만드는 바람직한 현상임

## 3) 이전학습
- 일반적인 vgg층 다음에 출력층을 추가하는 이전학습

In [5]:
from tensorflow.keras.applications import VGG16  # vgg의 마지막 부분을 내 데이터에 맞게 고친다!! >> 오버피팅을 막자
vgg_base=VGG16(weights='imagenet',include_top=False, input_shape=(150,150,3)) 
vgg_base.summary()  # 이미 training 되어있음

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
______________________________________________________________

In [14]:
# image data generator로 데이터 준비
import numpy as np
imagegen=ImageDataGenerator(rescale=1./255)  # 자료증대는 하지않음
batch = 20

def extract_features(directory,sample):
    features=np.zeros(shape=(sample,4,4,512))  # vgg의 output만큼 0
    labels=np.zeros(shape=(sample))

    # flow_from_directory
    generator=imagegen.flow_from_directory(directory,target_size=(150,150), batch_size=batch, class_mode='binary')  
    i=0

    # generator를 이용할 때는 배치단위이기 때문에 이렇게 vgg input을 받아야한다!
    for batched_input, batched_label in generator:  # 각 배치별로 input, output 출력
        
        batched_features=vgg_base.predict(batched_input)  # 기존 vgg에 통과시킨다음 predict값 저장
        
        # vgg predict값을 넣음
        features[ i*batch : (i+1)*batch]=batched_features
        labels[ i*batch : (i+1)*batch]=batched_label
        i+=1

        if i*batch>=sample:
            break

    return features, labels

In [25]:
train_input, train_labels=extract_features(train_dir,2000)  # 배치 20개씩 100번 반복
validation_input, validation_labels=extract_features(validation_dir,1000)
test_input, test_labels=extract_features(test_dir,1000)  # 배치 20개씩 50번 반복
train_input.shape

VGG 뒤에 일반 이전학습 과정으로 MLP층과 출력층 추가

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import optimizers

model1=Sequential()

### 추가할 부분
model1.add(Dense(256, activation='relu',input_dim=4*4*512))  # 너무 모수가 많기에 병목현상이 일어날 가능성이 크다
model1.add(Dropout(0.5))  # 따라서 드랍아웃을 시킴
model1.add(Dense(1, activation='sigmoid'))

model1.compile(optimizer=optimizers.RMSprop(lr=2e-5),loss='binary_crossentropy',metrics=['acc'])
model1_result = model1.fit(train_features,train_labels, epochs=20,batch_size=20,validation_data=(validation_features,validation_labels))

## 초기치가 랜덤이기 때문에, 초기치에 따라 수렴 속도가 달라진다. 껐다 다시돌렸는데 퍼포먼스가 많이 차이가 난다면 함수가 복잡하다는 뜻. 
## 따라서 에폭을 많이 돌려야함. 

과적합문제가 많이 완화된 것을 볼 수 있다!

In [ ]:
import matplotlib.pyplot as plt
acc=model1_result.history['acc']
val_acc=model1_result.history['val_acc']
loss=model1_result.history['loss']
val_loss=model1_result.history['val_loss']
epochs=range(1,len(acc)+1)
plt.plot(epochs,acc,'b',label='training acc')
plt.plot(epochs,val_acc,'bo',label='validation acc')
plt.title('training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs,loss,'b',label='training loss')
plt.plot(epochs,val_loss,'bo',label='validation loss')
plt.title('training and validation loss')
plt.legend()
plt.show()

## 4) 이전학습 + 자료증대
- VGG의 일부 모수는 고정하고, 새로운 은닉층을 추가하여 추가된 은닉층의 모수를 추정한다   
(예전에는 그냥 VGG에 통과시킨 결과를 다른 모델에 넣었음)
> 이 방법의 장점은 자료증대롤 사용할 수 있다는 것

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
model_aug=Sequential()

######### 우선 vgg 구조를 모델에 추가하기 ######
model_aug.add(vgg_base)  # 여기에 아예 집어넣음 >> 근데 여기 파라미터는 freez 시켜야함.(아니면 2000개의 파라미터를 가지고 1400만개의 파라미터를 추정하는 꼴이 됨) 그럼 위 과정과 완전히 같아진다. 
model_aug.add(Flatten())
model_aug.add(Dense(256,activation='relu'))
model_aug.add(Dense(1,activation='sigmoid'))  # 드랍아웃을 안했기때문에 위 결과보다 나쁠수밖에 없음. 
model_aug.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


15개의 가중치그룹과 bias 그룹의 모수를 따로 추정하므로 이들을 모두 다하면 30이 됨.     
 (15개 = VGG16 13개 + 추가한 MLP 2개)

In [17]:
print(len(model_aug.trainable_weights)) 

30


In [18]:
# VGG의 모수가 다시 train 되지 않게 고정해야한다!!
vgg_base.trainable=False  

# ImageDataGenerator > 자료증대
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=40, width_shift_range=0.2,
                               height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                horizontal_flip=True, fill_mode='nearest')  # train data의 자료를 늘린다(늘리고 엎고,,이런느낌)

validation_datagen=ImageDataGenerator(rescale=1./255)   #검증데이터는 자료증대를 하지 않아야 함. 리스케일만 함


# flow_from_directory
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(150,150),
                                              batch_size=20, class_mode='binary')
validation_generator=train_datagen.flow_from_directory(validation_dir,target_size=(150,150),
                                              batch_size=20, class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1010 images belonging to 2 classes.


In [23]:
model_aug.compile(optimizer=optimizers.RMSprop(lr=2e-5),loss='binary_crossentropy',metrics=['acc'])
model_aug_result=model_aug.fit_generator(train_generator,steps_per_epoch=200, epochs=20, validation_data=validation_generator,validation_steps=50)

과대적합은 어느정도 해결되고 있지만, 자료증대로 인해 노이즈가 추가되어 정밀도는 낮아진다. 




## 5) Fine-Tuning
- 지금까지 VGG층은 건든 적이 없지만(freeze), 미세조정으로 VGG의 conv 층을 trainable하게 바꾸고 그 뒤에 MLP까지 추가하여 새로운 맞춤형 모형을 만든다

In [24]:
vgg_base.trainable=True   # vgg 모수를 trainable하게 바꾼다!


trainable_layer=False  

# 레이어를 읽기 시작
for layer in vgg_base.layers:  
    if layer.name=='block5_conv1': # block5_conv1부터 trainable_layer가 True로 layer.trainable이 True로 바뀜
        trainable_layer=True  # 쟤가 읽혀지면 True로 바꾸기. 그 다음부터는 True로 하고 trainiable하라는 뜻!

    if trainable_layer:
        layer.trainable=True 

    else:
        layer.trainable=False

In [ ]:
from tensorflow.keras import optimizers
model_aug.compile(optimizer=optimizers.RMSprop(lr=1e-5),loss='binary_crossentropy',metrics=['acc'])

# 앞서 사용한 자료증대 + 이전학습 모형을 그대로 가져온다. 근데 단, conv net이 trainable된
result_final=model_aug.fit_generator(train_generator,
                                     steps_per_epoch=200, 
                                     epochs=30, 
                                     validation_data=validation_generator,
                                     validation_steps=50)

미세조정결과 성능이 증가한다!

In [ ]:
import matplotlib.pyplot as plt
acc=result_final.history['acc']
val_acc=result_final.history['val_acc']
loss=result_final.history['loss']
val_loss=result_final.history['val_loss']
epochs=range(1,len(acc)+1)
plt.plot(epochs,acc,'b',label='training acc')
plt.plot(epochs,val_acc,'bo',label='validation acc')
plt.title('training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs,loss,'b',label='training loss')
plt.plot(epochs,val_loss,'bo',label='validation loss')
plt.title('training and validation loss')
plt.legend()
plt.show()

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_generator=test_datagen.flow_from_directory(test_dir,target_size=(150,150),
                                              batch_size=20, class_mode='binary')
loss, acc=model_aug.evaluate_generator(test_generator,steps=50)
print(loss)
print(acc)  # 2000개의 작은 데이터를 가지고 이전학습을 사용